In [62]:
from tqdm import tqdm
import pandas as pd
import copy
import datetime
import sys
import imp

In [63]:
utils = imp.load_source('utils','/home/alifer/PycharmProjects/227/congresso-em-numeros/utils.py')

class Capture():

    def __init__(self,house=None,api_type=None,api_name=None,table=None,schema=None,entry_columns=[]):
        """ Parent class for data capture.
        :param house: str
        :param api_type: str
        :param api_name: str
        :param table: str
        :param schema: str
        :param entry_columns: list
        """
        self.house = house
        self.api_type = api_type
        self.api_name = api_name
        self.schema = schema
        self.entry_columns = dict.fromkeys(entry_columns)

        if table:
            self.table = table
        else:
            self.table = ('_').join([house, api_type, api_name])

        self.entry_structure = dict.fromkeys(entry_columns)
        self.conn = utils.connect_sqlalchemy()

        self.conn_psc2 = utils.connect_psycopg2()
        self.cursor = self.conn_psc2.cursor()
        sql = "set datestyle to SQL,DMY;"
        self.cursor.execute(sql)

    def daterange(self,start_date,end_date):
        for n in range(int((start_date - end_date).days)):
            yield start_date - datetime.timedelta(n)

In [4]:
class Especies_legislativas(Capture):
    
    def __init__(self,house=None,api_type=None,api_name=None,table=None,schema=None,entry_columns=None):
    #     Capture.__init__(self,house,api_type,api_name,table,schema,entry_columns)
        self.entry_columns = entry_columns
            
    def fill_entry_especie_legislativa(self,materia,entry_columns):
        tgd = utils.try_get_data
        
        IdentificacaoMateria = materia['IdentificacaoMateria']
        DadosBasicosMateria = materia['DadosBasicosMateria']
        
        id_materia = IdentificacaoMateria['CodigoMateria']
        casa_materia = IdentificacaoMateria['NomeCasaIdentificacaoMateria']
        id_especie_legislativa = utils.generate_hash(id_materia, casa_materia)

        dados_materia = utils.get_json('http://legis.senado.leg.br/dadosabertos/materia/{}'.format(id_materia))
        OrigemMateria = tgd(dados_materia['DetalheMateria'],'OrigemMateria')
        CasaIniciadora = tgd(dados_materia['DetalheMateria'],'CasaIniciadoraNoLegislativo')
        
        entry = copy.deepcopy(entry_columns)
        entry['id_especie_legislativa'] = id_especie_legislativa
        entry['data_captura'] = datetime.datetime.today()
        entry['nome_casa_identificacao'] = casa_materia
        entry['id_casa_original'] = id_materia
        
        entry['sigla_casa_identificacao'] = tgd(IdentificacaoMateria,'SiglaCasaIdentificacaoMateria')
        entry['descricao_subtipo'] = tgd(IdentificacaoMateria,'DescricaoSubtipoMateria')
        entry['sigla_subtipo'] = tgd(IdentificacaoMateria,'SiglaSubtipoMateria')
        entry['numero'] = tgd(IdentificacaoMateria,'NumeroMateria')
        entry['ano'] = tgd(IdentificacaoMateria,'AnoMateria')
        entry['indicador_tramitando'] = tgd(IdentificacaoMateria,'IndicadorTramitando')
        
        entry['ementa'] = tgd(DadosBasicosMateria,'EmentaMateria')
        entry['explicacao_ementa'] = tgd(DadosBasicosMateria,'ExplicacaoEmentaMateria')
        entry['obsercacao'] = tgd(DadosBasicosMateria,'ObservacaoMateria')
        entry['apelido'] = tgd(DadosBasicosMateria,'ApelidoMateria')
        entry['indexacao'] = tgd(DadosBasicosMateria,'IndexacaoMateria')
        entry['indicador_complementar'] = tgd(DadosBasicosMateria,'IndicadorComplementar')
        entry['data_apresentacao'] = tgd(DadosBasicosMateria,'DataApresentacao')
        entry['data_leitura'] = tgd(DadosBasicosMateria,'DataLeitura')
        entry['sigla_casa_leitura'] = tgd(DadosBasicosMateria,'SiglaCasaLeitura')
        entry['nome_casa_leitura'] = tgd(DadosBasicosMateria,'NomeCasaLeitura')
        
        if OrigemMateria:
            entry['nome_poder_origem'] = tgd(OrigemMateria,'NomePoderOrigem')
            entry['sigla_casa_origem'] = tgd(OrigemMateria,'SiglaCasaOrigem')
            entry['nome_casa_origem'] = tgd(OrigemMateria,'NomeCasaOrigem')
            
        if CasaIniciadora:
            entry['sigla_casa_iniciadora'] = tgd(CasaIniciadora,'SiglaCasaIniciadora')
            entry['nome_casa_iniciadora'] = tgd(CasaIniciadora,'NomeCasaIniciadora')

        return entry

In [68]:
cap = Especies_legislativas(table = 'a',entry_columns=entry_columns)

In [70]:
df = cap.fill_entry_especie_legislativa(materia,entry_columns)
pd.DataFrame([df])

,ano,apelido,data_apresentacao,data_captura,data_leitura,descricao_subtipo,ementa,explicacao_ementa,id_casa_original,id_especie_legislativa,...,nome_casa_leitura,nome_casa_origem,nome_poder_origem,numero,obsercacao,sigla_casa_identificacao,sigla_casa_iniciadora,sigla_casa_leitura,sigla_casa_origem,sigla_subtipo
0,1990,None,1990-04-30,2017-07-07,1990-04-30,ANTEPROJETO DE LEI DO DF,DISPÕE SOBRE A CRIAÇÃO DE QUADRO DE PESSOAL DA...,Visa declarar Nilo Peçanha Patrono da Educação...,2073,2fab9a81767f57fce77cde47c08d37d5,...,None,None,None,00001,None,SF,None,None,None,ADF


In [45]:
def fill_entry_parlamentar_autoria(self, autoria, entry_columns):
    tgd = utils.try_get_data
    
    IdentificacaoMateria = autoria['Materia']['IdentificacaoMateria']
    
    id_materia = IdentificacaoMateria['CodigoMateria']
    casa_materia = IdentificacaoMateria['NomeCasaIdentificacaoMateria']
    id_especie_legislativa = utils.generate_hash(id_materia, casa_materia)

    entry = copy.deepcopy(entry_columns)
    entry['id_especie_legislativa'] = id_especie_legislativa
    entry['indicador_autor_principal'] = tgd(parlamentar,'IndicadorAutorPrincipal')
    entry['numero_ordem_autor'] = tgd(parlamentar,'NumeroOrdemAutor')
    entry['indicador_outros_autores'] = tgd(parlamentar,'IndicadorOutrosAutores')
    
    return entry

In [89]:
url = 'http://legis.senado.leg.br/dadosabertos/senador/70/mandatos'
data_source = utils.get_json(url)

In [94]:
data_source['MandatoParlamentar']['Parlamentar']['Mandatos']['Mandato'][1]

{'CodigoMandato': '287',
 'DescricaoParticipacao': 'Titular',
 'Exercicios': {'Exercicio': {'CodigoExercicio': '1720',
   'DataFim': '2011-01-31',
   'DataInicio': '2003-02-01',
   'DescricaoCausaAfastamento': 'Término do mandato',
   'SiglaCausaAfastamento': 'TER'}},
 'PrimeiraLegislaturaDoMandato': {'DataFim': '2007-01-31',
  'DataInicio': '2003-02-01',
  'NumeroLegislatura': '52'},
 'SegundaLegislaturaDoMandato': {'DataFim': '2011-01-31',
  'DataInicio': '2007-02-01',
  'NumeroLegislatura': '53'},
 'Suplentes': {'Suplente': [{'CodigoParlamentar': '4843',
    'DescricaoParticipacao': '1º Suplente',
    'NomeParlamentar': 'José Oliveira Costa'},
   {'CodigoParlamentar': '3613',
    'DescricaoParticipacao': '2º Suplente',
    'NomeParlamentar': 'José Wanderley'}]},
 'UfParlamentar': 'AL'}

In [95]:
data_source['MandatoParlamentar']['Parlamentar']['Mandatos']['Mandato'][2]

{'CodigoMandato': '4',
 'DescricaoParticipacao': 'Titular',
 'Exercicios': {'Exercicio': [{'CodigoExercicio': '764',
    'DataFim': '2003-01-31',
    'DataInicio': '1999-07-19',
    'DescricaoCausaAfastamento': 'Término do mandato',
    'SiglaCausaAfastamento': 'TER'},
   {'CodigoExercicio': '96',
    'DataFim': '1998-04-06',
    'DataInicio': '1995-02-01',
    'DataLeitura': '1998-04-07',
    'DescricaoCausaAfastamento': 'Afastamento do exercício',
    'SiglaCausaAfastamento': 'AFO'}]},
 'PrimeiraLegislaturaDoMandato': {'DataFim': '1999-01-31',
  'DataInicio': '1995-02-01',
  'NumeroLegislatura': '50'},
 'SegundaLegislaturaDoMandato': {'DataFim': '2003-01-31',
  'DataInicio': '1999-02-01',
  'NumeroLegislatura': '51'},
 'Suplentes': {'Suplente': {'CodigoParlamentar': '96',
   'DescricaoParticipacao': '1º Suplente',
   'NomeParlamentar': 'Djalma Falcão'}},
 'UfParlamentar': 'AL'}

In [ ]:

        try:
            iterable = data_source['ListaVotacoes']['Votacoes']['Votacao']
            cap.entry_columns['url_captura'] = url
            cap.update_DB(iterable=iterable, entry_columns=cap.entry_columns)
        except KeyError:
            pass

In [111]:
a = utils.get_json('http://legis.senado.leg.br/dadosabertos/plenario/lista/votacao/20160401/20160430')

In [115]:
a['ListaVotacoes']['Votacoes']['Votacao']

[{'AnoMateria': '2015',
  'CodigoMateria': '119852',
  'CodigoSessao': '22500',
  'CodigoSessaoLegislativa': '853',
  'CodigoSessaoVotacao': '5549',
  'CodigoTramitacao': '2337452',
  'DataSessao': '2016-04-06',
  'DescricaoVotacao': 'Votação nominal da Emenda nº 6-Plen (Substitutivo) ao PLS 78/2015.',
  'HoraInicio': '14:00',
  'NumeroMateria': '00078',
  'NumeroSessao': '45',
  'Resultado': 'A',
  'Secreta': 'N',
  'SequencialSessao': '1',
  'SiglaCasa': 'SF',
  'SiglaMateria': 'PLS',
  'TipoSessao': 'DOR',
  'Votos': {'VotoParlamentar': [{'CodigoParlamentar': '40',
     'Foto': 'http://www.senado.leg.br/senadores/img/fotos-oficiais/senador40.jpg',
     'NomeParlamentar': 'José Agripino',
     'SexoParlamentar': 'M',
     'SiglaPartido': 'DEM',
     'SiglaUF': 'RN',
     'Tratamento': 'Senador',
     'Url': 'http://www25.senado.leg.br/web/senadores/senador/-/perfil/40',
     'Voto': 'Sim'},
    {'CodigoParlamentar': '16',
     'Foto': 'http://www.senado.leg.br/senadores/img/fotos-ofi

In [9]:
legislatura = a['ListaVotacoes']['Votacoes']['Votacao'][5]

NameError: name 'a' is not defined

In [ ]:
legislatura

In [65]:
data_sessao = legislatura['dataSessao']

In [97]:
for date in generate_date_range(start,end):
    d = date.strftime('%Y%m%d')
    dic = utils.get_json('http://legis.senado.leg.br/dadosabertos/plenario/resultado/{}'.format(d))
    try:
        sessao = dic['ResultadoPlenario']['Sessoes']['Sessao']['comunicado']
        print(d)
    except (KeyError, TypeError):
        try:
            sessao = dic['ResultadoPlenario']['Sessoes']['Sessao']['Comunicado']
            print(d)
        except (KeyError, TypeError):
            pass

In [38]:
def generate_date_range(start_date,end_date):
    """
    Generate a range of dates between start_date and end_date

    :param start_date: date object
    :param end_date: date object
    :return: list -> list of datetime objects
    """
    day = start_date
    days_delta = (end_date - start_date).days
    for i in range(days_delta + 1):
        yield day
        day += datetime.timedelta(1)

In [43]:
start

datetime.datetime(2010, 1, 1, 0, 0)

In [46]:
for start.strftime('%Y%m%d'):
    

'20100101'

In [1]:
def convert_str_date(date,pattern1,pattern2):
    dt = datetime.datetime.strptime(date,pattern1).date()
    dt = dt.strftime(pattern2)
    return dt

In [6]:
convert_str_date('10-15-2012', '%d-%m-%Y', '%Y%m%d')

ValueError: time data '10-15-2012' does not match format '%d-%m-%Y'

In [14]:
a = "asas \
        bb"

In [15]:
a

'asas         bb'